In [31]:
# 打印所有单行变量
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

> 个人理解:  
> python一切皆对象，可以分为静态对象和动态对象。  
> 静态对象：数字，字符串(现实中的大小，颜色等)  
> 动态对象：函数(现实中各种动作，扔东西会下落，打玻璃会碎等)  

既然函数也是对象，所以函数作为其他函数的参数，或者被return  
>前面经常用的dir()和help()就是把函数对象作为参数

## 闭包

> 在一些语言中，在函数中可以（嵌套）定义另一个函数时，如果内部的函数引用了外部的函数的变量，则可能产生闭包。

In [32]:
abs(-10)  # -->
abs       # abs(-10)是函数调用，而abs是函数本身

10

<function abs>

In [33]:
abs.__name__  # -->
f = abs       # 对象指向f标签
f             # -->
f(-10)        # -->
f.__name__    # --> 本质还是原来的abs对象

'abs'

<function abs>

10

'abs'

In [34]:
def foo(a, b, c):
    def bar(x):
        return a*x**2 + b*x + c  # 向上层作用域搜索a,b,c
    return bar                   # 返回一个函数对象

y = foo(2, 3, 4)

y            # --> 
y(6)  # -->

<function __main__.foo.<locals>.bar>

94

In [35]:
y.__closure__                   # --> cell对象的元组
y.__closure__[0].cell_contents  # --> 每个cell对象携带着外部参数
y.__closure__[1].cell_contents  # -->
y.__closure__[2].cell_contents  # -->

(<cell at 0x7f04bc298be8: int object at 0x559466dd3f60>,
 <cell at 0x7f04bc298648: int object at 0x559466dd3f80>,
 <cell at 0x7f04bc298e58: int object at 0x559466dd3fa0>)

2

3

4

## Decorator(装饰器）

> 装饰模式有很多经典的使用场景，例如插入日志、性能测试、事务处理等等，有了装饰器，就可以提取大量函数中与本身功能无关的类似代码，从而达到代码重用的目的

装饰器通常用于在不改变原有函数代码和功能的情况下，为其添加额外的功能

In [36]:
def hello():                # 创建基础函数
    return 'hello world'

def foo(func):
    def bar():
        return "<i>" + func() + "</i>"  # 调用形参
    return bar                          # 返回函数对象

hello.__name__      # -->
hello = foo(hello)  # hello变量指向了foo.<locals>.bar
hello.__name__      # -->
hello()             # --> 本质是调用了foo.<locals>.bar

'hello'

'bar'

'<i>hello world</i>'

### @ 语法糖

使用装饰器提供的 @ 语法糖（Syntactic Sugar）简化代码:

```
@decorator  # <=> func = decorator(func)
def func():
    pass

```

In [37]:
def foo(func):
    def bar():
        return "<i>" + func() + "</i>"
    return bar

@foo          # 语法糖会处理最近的函数等效与hello = foo(hello)
def hello():
    return 'hello world'

hello           # -->
hello.__name__  # --> 这里的hello只是一个指向foo.<locals>.bar函数的变量
hello()         # -->

<function __main__.foo.<locals>.bar>

'bar'

'<i>hello world</i>'

### 对带参数的函数进行装饰

In [38]:
def foo(func):
    def bar(*arg, **kwargs):
        ret = func(*arg, **kwargs)
        return "<i>" + ret + "</i>"
    return bar

@foo
def hello(name):
    return 'hello {}'.format(name)

@foo
def say(name1, name2):
    return 'hello {0} and {1}'.format(name1,name2)

hello('python')            # -->
say('python', 'markdwon')  # -->

'<i>hello python</i>'

'<i>hello python and markdwon</i>'

### 带参数的装饰器

带参数的装饰器
```
@foo(arg1, arg2)  # <=> func = foo(arg1, arg2)(func)
def func():
    pass
```

In [39]:
def bar_tags(tag):
    def foo(func):
        def bar(*arg, **kwargs):
            ret = func(*arg, **kwargs)
            return '<' + tag + '>' + ret + '</' + tag + '>'
        return bar
    return foo

@bar_tags('b')  # 根据不同参数返回不同的装饰器
def hello(name):
    return 'hello {}'.format(name)

hello('python')

'<b>hello python</b>'

### 多个装饰器

装饰器可以定义多个，离函数定义最近的装饰器先被调用

```
@foo      # <=> func = foo(bar(func))
@bar
def func():
    pass
```

In [40]:
def foo(func):
    def bar():
        return "<i>" + func() + "</i>"
    return bar

def add_tag(func):
    def bar():
        return '<b>' + func() + '</b>'
    return bar

@foo
@add_tag
def hello():
    return 'hello world'

hello()

'<i><b>hello world</b></i>'

### 基于类的装饰器

In [41]:
class Bold(object):
    def __init__(self, func):
        self.func = func

    def __call__(self, *args, **kwargs):
        return '<b>' + self.func(*args, **kwargs) + '</b>'

@Bold
def hello(name):
    return 'hello %s' % name

hello('world')

'<b>hello world</b>'

类 Bold 有两个方法：
* `__init__()`：它接收一个函数作为参数，也就是被装饰的函数  
* `__call__()`：让类对象可调用，就像函数调用一样，在调用被装饰函数时被调用

> 如果类装饰器有参数，则 `__init__ `接收参数，而 `__call__` 接收 func

> 使用装饰器有一个瑕疵，就是被装饰的函数，它的函数名称已经不是原来的名称,为了消除这样的副作用，Python 中的 functool 包提供了一个 wraps 的装饰器

In [44]:
class Tag(object):
    def __init__(self, tag):
        self.tag = tag

    def __call__(self, func):
        def wrapped(*args, **kwargs):
            return "<{tag}>{res}</{tag}>".format(
                res=func(*args, **kwargs), tag=self.tag
            )
        return wrapped

@Tag('i')
def hello(name):
    return 'hello %s' % name

hello('world')

'<i>hello world</i>'